<a href="https://colab.research.google.com/github/adnansee/PythonTensorflow2.0/blob/master/MakeMultipleColumnsFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q sklearn
!pip install -q category-encoders
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher
import category_encoders as ce

# LOAD DATA IN CSV FORM (FOR OTHER FORMS CHANGE PANDAS METHOD) - ALSO CLEAN THE COLUMN NAMES AND REMOVE 'UNNAMED' COLUMNS - VISULIZE

URL = 'https://raw.githubusercontent.com/adnansee/csvdata/master/Workinghours(1).csv'
dataframe = pd.read_csv(URL, header=0)
dataframe = dataframe.loc[:, ~dataframe.columns.str.contains('^Unnamed')]
dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
dataframe.head(5)
dataframe.education = dataframe.education.astype('float64')
dataframe2 = dataframe.copy()

allDataFramesAndSets = []
finalModel = []
allFeatures = []
dataframe.head()

,hours,income,age,education,child5,child13,child17,nonwhite,owned,mortgage,occupation,unemp
0,2000,350,26,12.0,0,1,0,0,1,1,swcc,7
1,390,241,29,8.0,0,1,1,0,1,1,other,4
2,1900,160,33,10.0,0,2,0,0,1,0,swcc,7
3,0,80,20,9.0,2,0,0,0,1,1,other,7
4,3177,456,33,12.0,0,2,0,0,1,1,swcc,7


In [0]:
def makeManyFeaturesColumns(dataframe, target, categoricalColumn):

    #WOE ENCODING
    woe_df = dataframe.groupby(categoricalColumn)[target].mean()
    woe_df = pd.DataFrame(woe_df)
    woe_df = woe_df.rename(columns  = {target : 'Good'})
    woe_df['Bad'] = 1 - woe_df.Good
    woe_df['Bad'] = np.where(woe_df['Bad'] == 0, 0.000001, woe_df['Bad'])
    woe_df['WoE'] = np.log(woe_df.Good / woe_df.Bad)
    dataframe.loc[:,'woe_'+categoricalColumn] = dataframe[categoricalColumn].map(woe_df['WoE'])

    #POE ENCODING
    poe_df = dataframe.groupby(categoricalColumn)[target].mean()
    poe_df = pd.DataFrame(poe_df)
    poe_df = poe_df.rename(columns  = {target : 'Good'})
    poe_df['Bad'] = 1 - poe_df.Good
    poe_df['Bad'] = np.where(poe_df['Bad'] == 0, 0.000001, poe_df['Bad'])
    poe_df['PoE'] = poe_df.Good / poe_df.Bad
    dataframe.loc[:,'poe_'+categoricalColumn] = dataframe[categoricalColumn].map(poe_df['PoE'])

    #MEAN ENCODING
    meanEncoder = dataframe.groupby(categoricalColumn)[target].mean()
    dataframe.loc[:,'mean_'+categoricalColumn] = dataframe[categoricalColumn].map(meanEncoder)

    #LABEL ENCODING
    labelEncoder = LabelEncoder()
    dataframe.loc[:,'label_'+categoricalColumn] = labelEncoder.fit_transform(dataframe[categoricalColumn])

    #FREQUENCY ENCODING
    frequencyEncoding = dataframe.groupby(categoricalColumn).size()/len(dataframe)
    dataframe.loc[:,'frequency_'+categoricalColumn] = dataframe[categoricalColumn].map(frequencyEncoding)

#BINARY ENCODING
def binaryEncoder(dataframe, categoricalColumn):
    ce_binary = ce.BinaryEncoder(cols = categoricalColumn)
    return ce_binary.fit_transform(dataframe[categoricalColumn])

# MOST OFTEN NOMINAL DATA
def oneHotEncoder(dataframe, categoricalColumn):
    columnName_bin = feature_column.categorical_column_with_vocabulary_list(
    categoricalColumn, dataframe[categoricalColumn].unique())
    columnName_bin_one_hot = feature_column.indicator_column(columnName_bin)
    return columnName_bin_one_hot
  
#ONE HOT EMBEDDING ENCODING
def oneHotEmbbEncoder(dataframe, categoricalColumn):
    columnName_bin = feature_column.categorical_column_with_vocabulary_list(
    categoricalColumn, dataframe[categoricalColumn].unique())
    columnName_bin_one_hot = feature_column.embedding_column(columnName_bin, dimension= (dataframe[categoricalColumn].value_counts().size))
    return columnName_bin_one_hot

#HASHED ENCODING
def hashedColumnEncoder(dataframe, categoricalColumn):
    hashed = feature_column.categorical_column_with_hash_bucket(
    categoricalColumn, hash_bucket_size=dataframe[categoricalColumn].value_counts().size)
    return hashed
    

#NORMALISING
def normalize(columnName, dataframe):
    x = dataframe[[columnName]].values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    # Run the normalizer on the dataframe
    dataframe[columnName] = x_scaled

#CREATE ONEHOT EMBEDDINGS
def createOneHotEmbeddingColumns(dataframe, categoricalColumn):
    ohc = oneHotEncoder(dataframe, categoricalColumn)
    ohec = oneHotEmbbEncoder(dataframe, categoricalColumn)
    hc = hashedColumnEncoder(dataframe, categoricalColumn)
    be = binaryEncoder(dataframe, categoricalColumn)
    embbOjtArray = [ohc,ohec,hc,be]
    print((ohc, ohec, hc, be))
    #allFeatures.extend((ohc, ohec, hc, be))
    return embbOjtArray


#CONVERT DATAFRAME TO DATASET

def df_to_dataset(dataframe, labelName, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop(labelName)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

                                                  
#FILL NAN WITH MEAN VALUES FOR NUMERICAL COLUMNS

def fillNanWithMean(column):
  #for column in dataframe.iteritems():
    dataframe[column].fillna((dataframe[column].mean()), inplace=True)


#MAKE ALL THE FEATURES
def makeAll(dataframe, target):
  
  
  for column in dataframe:
    if dataframe[column].dtype == 'object': 
      originalDtype = dataframe[column].dtype
      makeManyFeaturesColumns(dataframe, target, column)
      #allFeatures.append((column))
      print(column, ' - ', len(dataframe[column].value_counts()),' different values and was of',originalDtype, 'type. FEATURE COLUMNS CREATED (POE - WOE - EMBB - LAB - MEAN - LABEL - FREQ)')
       
  for column in dataframe: 
    if dataframe[column].dtype == 'int64' or dataframe[column].dtype == 'float64':
      originalDtype = dataframe[column].dtype
      if len(dataframe[column].value_counts()) > 2:
        normalize(column, dataframe)
        print(column, ' - ', len(dataframe[column].value_counts()),' different values and of',originalDtype, 'type was normalised ')
  

  allFeatures.extend((dataframe.columns.values))
  allFeatures.remove(target)

 # allFeatures.extend((dataframe.columns.values))
  for column in dataframe:
    if dataframe[column].dtype == 'object': 
      allFeatures.remove(column)     

print(allFeatures)
# SPLIT THE DATA
def splitDataFrame(dataframe, batch_size, label):
  train, test = train_test_split(dataframe, test_size=0.2)
  train, val = train_test_split(train, test_size=0.2)
  print(len(train), 'train examples')
  print(len(val), 'validation examples')
  print(len(test), 'test examples')
  
  train_ds = df_to_dataset(train, label, batch_size=batch_size)
  val_ds = df_to_dataset(val, label, shuffle=False, batch_size=batch_size)
  test_ds = df_to_dataset(test, label, shuffle=False, batch_size=batch_size)

  trainTestData = [train_ds, val_ds, test_ds]
  allDataFramesAndSets.extend((train_ds, val_ds, test_ds, train, test))
  return trainTestData

# MODEL COMPILE AND FIT **************************************************************************************************************************

# CREATE THE FEATURE LAYER WITH THE UPDATED COLUMNS
def compileFitModel(dataframe, label):
  feature_columns = []
  #allFeatures.remove(label)
  
  for header in allFeatures:#['income','age','child13','unemp','woe_occupation','nonwhite','hours', 'education', 'owned','child5','child17',]:
    feature_columns.append(feature_column.numeric_column(header))
  feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

  feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

  # ADD OTHER LAYERS DEPENDING ON MODEL
  model = keras.Sequential([
      feature_layer,  # input layer (1)
      keras.layers.Dense(128, activation='relu'),  # hidden layer (2)
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dense(1) # output layer (3)
      #keras.layers.Dense(4, activation='softmax')
  ])

  # COMPILE MODEL
  model.compile(optimizer='adam',
                  #loss='sparse_categorical_crossentropy',
                  #loss=tf.keras.losses.CategoricalCrossentropy(),
                  #loss=tf.keras.losses.KLDivergence(),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  #loss=tf.keras.losses.MeanAbsoluteError(),
                  #loss=tf.keras.losses.Poisson(),
                  metrics=['accuracy'])

  #GET THE DATA
  dataframeArray = splitDataFrame(dataframe, 32, label)
  
  

  
  # FIR MODEL
  modelHis = model.fit((dataframeArray[0]),
          validation_data=(dataframeArray[1]),
          epochs=10)  # we pass the data, labels and epochs and watch the magic!
  finalModel.append(model)
  finalModel.append(modelHis)

  return model    

#                                                               SHOW HIT RESULTS IN PERCENTAGE

#????????????????????????????????

def showHitResult(test_ds, label, test, model):

  labels = pd.DataFrame(test[label].astype('float64'))
  
  
  results = model.predict(test_ds)
  results = pd.DataFrame(results)
  results[results>=0.5]=1
  results[results<0.5]=0
  
  resultsArray = results.rename(columns={0:label})
  labels = labels.reset_index(drop=True)
  labelsFloat = test[label].astype('float64')
  resultsArray.set_index(label)

  print('Real target results')
  print(labels)
  print('Predicted results relow')
  print(resultsArray)

  A = resultsArray
  B = labels
  number_of_equal_elements = np.sum(A==B)
  total_elements = np.multiply(*A.shape)
  percentage = number_of_equal_elements/total_elements
  print(percentage*100)


  print('total number of elements: \t\t\t{}'.format(total_elements))
  print('number of identical elements: \t\t{}'.format(number_of_equal_elements))
  print('number of different elements: \t\t{}'.format(total_elements-number_of_equal_elements))
  print((percentage)*100.0)
  #print(dr.head(10))

  

#METHOD TO SHOW ALL CATEGORIES FOR EVERY COLUMN IN THE DATAFRAME
def showAllCategories(dataframe):
    for column in dataframe:
      print(dataframe[column].value_counts())

#print('Train: %.3f, Test: %.3f' % (len(train_ds), len(test_ds))
#PLOT THE LOSS AND ACCURACY
def plotLossAccuracy(model):
  plt.title('Loss')
  plt.plot(model.history['loss'], label='Train')
  plt.plot(model.history['val_loss'], label='Test')
  plt.legend()
  plt.show()
  plt.title('Accuracy')
  plt.plot(model.history['accuracy'], label='Train')
  plt.plot(model.history['val_accuracy'], label='Test')
  plt.legend()
  plt.show()



['hours', 'income', 'age', 'education', 'child5', 'child13', 'child17', 'nonwhite', 'owned', 'unemp', 'woe_occupation', 'poe_occupation', 'mean_occupation', 'label_occupation', 'frequency_occupation']


In [0]:
makeAll(dataframe, 'mortgage')
onehotfeatures = createOneHotEmbeddingColumns(dataframe, 'occupation')


In [0]:
compileFitModel(dataframe, 'mortgage')

In [0]:
finalModel

In [0]:
plotLossAccuracy(finalModel[1])

#  TEST ACCURACY
loss, accuracy = finalModel[0].evaluate(allDataFramesAndSets[2])
print("Accuracy", accuracy * 100)

showHitResult(allDataFramesAndSets[2],'mortgage',allDataFramesAndSets[4], finalModel[0])

finalModel[0].predict(allDataFramesAndSets[0])



IndexError: ignored

In [0]:
print(finalModel[0])

In [0]:
print(allFeatures)

['hours', 'income', 'age', 'education', 'child5', 'child13', 'child17', 'nonwhite', 'owned', 'unemp', 'poe_occupation']


In [0]:
allFeatures.remove('mean_occupation')
allFeatures.remove('woe_occupation')
allFeatures.remove('label_occupation')
allFeatures.remove('frequency_occupation')


In [0]:
dataframe.head()

In [0]:
finalModel[0].predict(allDataFramesAndSets[0])


In [0]:
print(dataframe.occupation.dtypes)
print(dataframe.dtypes)
print(dataframe.head(5))
#dataframe.hours = dataframe.hours.astype('float64')